# Stock Analysis and Sentiment Correlation Dashboard
This notebook performs:
- Technical analysis using TA-Lib and PyNance
- Visualization of indicators
- Correlation analysis between news sentiment and stock returns

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Task 2: Load and Prepare Financial Data

In [ ]:
import pandas as pd
import numpy as np
import talib
import matplotlib.pyplot as plt

# Load stock data
goog_df = pd.read_csv('/content/GOOG_historical_data.csv', parse_dates=['Date'])
msft_df = pd.read_csv('/content/MSFT_historical_data.csv', parse_dates=['Date'])
amzn = pd.read_csv('/content/AMZN_historical_data.csv', parse_dates=['Date'])
nvda = pd.read_csv('/content/NVDA_historical_data.csv', parse_dates=['Date'])
meta = pd.read_csv('/content/META_historical_data.csv', parse_dates=['Date'])
tsla = pd.read_csv('/content/TSLA_historical_data.csv', parse_dates=['Date'])
# Sort by date and reset index
goog_df.sort_values('Date', inplace=True)
msft_df.sort_values('Date', inplace=True)
goog_df.reset_index(drop=True, inplace=True)
msft_df.reset_index(drop=True, inplace=True)

# Preview data
goog_df.head()

## Calculate Technical Indicators with TA-Lib

In [ ]:
# Add technical indicators
goog_df['SMA_20'] = talib.SMA(goog_df['Close'], timeperiod=20)
goog_df['RSI'] = talib.RSI(goog_df['Close'], timeperiod=14)
macd, macdsignal, macdhist = talib.MACD(goog_df['Close'])
goog_df['MACD'] = macd
goog_df['MACD_Signal'] = macdsignal

goog_df[['Date', 'Close', 'SMA_20', 'RSI', 'MACD', 'MACD_Signal']].tail()

## Visualize Technical Indicators

In [ ]:
# Plotting SMA and Close prices
plt.figure(figsize=(12, 6))
plt.plot(goog_df['Date'], goog_df['Close'], label='Close Price')
plt.plot(goog_df['Date'], goog_df['SMA_20'], label='SMA 20')
plt.title('GOOG Close Price and SMA')
plt.legend()
plt.show()

## Task 3: Correlation Between News and Stock Movements

In [ ]:
from textblob import TextBlob

# Example news data (in real scenario, load your own dataset)
news_data = pd.DataFrame({
    'Date': pd.date_range(start='2023-01-01', periods=10, freq='D'),
    'Headline': [
        'Google launches new AI tool',
        'Microsoft acquires new company',
        'Google faces antitrust lawsuit',
        'Microsoft reports record earnings',
        'Google announces layoffs',
        'Microsoft launches Copilot update',
        'Google AI outperforms expectations',
        'Microsoft cloud sees growth',
        'Google stock surges after earnings',
        'Microsoft faces regulation scrutiny'
    ]
})

# Sentiment Analysis
news_data['Sentiment'] = news_data['Headline'].apply(lambda x: TextBlob(x).sentiment.polarity)
news_data['Date'] = pd.to_datetime(news_data['Date'])
news_data.head()

### Compute Daily Returns for Stock

In [ ]:
goog_df['Return'] = goog_df['Close'].pct_change()
goog_df['Date'] = pd.to_datetime(goog_df['Date'])
goog_df_return = goog_df[['Date', 'Return']].dropna()
goog_df_return.head()

### Correlation Between Sentiment and Stock Return

In [ ]:
# Aggregate news sentiment by date
daily_sentiment = news_data.groupby('Date')['Sentiment'].mean().reset_index()

# Merge with stock returns
merged = pd.merge(goog_df_return, daily_sentiment, on='Date', how='inner')

# Calculate correlation
correlation = merged['Return'].corr(merged['Sentiment'])
print(f'Pearson correlation between sentiment and stock return: {correlation:.4f}')
merged